In [48]:
#Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



# Rest
from bs4 import BeautifulSoup
import time
import pandas as pd
import datetime
import h5py
import numpy as np

# Global settings for the driver
chrome_options = Options()
chrome_options.add_experimental_option("detach", True) # keeps driver open until manual termination

In [2]:
def launch_chrome(url):
    '''A method to install the latest Chrome driver and get url
    ---------
    Parameters:
    url = url defined by user'''
    
    global driver
    # installing driver making sure that new driver is up to date
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    return driver

In [13]:
launch_chrome('https://coinmarketcap.com/currencies/xrp/project-info/')

<selenium.webdriver.chrome.webdriver.WebDriver (session="122c9acddac91163a64620249511d824")>

In [37]:
top_crypto = [
    'bitcoin', 'ethereum', 'tether', 'usd-coin', 'bnb', 'xrp', 'binance-usd',
    'cardano', 'solana', 'dogecoin', 'polkadot-new', 'polygon', 'tron', 'HEX'
]

crypto_project_info = []

for crypto in top_crypto:
    launch_chrome(f'https://coinmarketcap.com/currencies/{crypto}/project-info/')
    
    # driver.maximize_window()
    
    # Cookie element blocks
    cookie = driver.find_element(By.XPATH, '//*[@id="cmc-cookie-policy-banner"]/div[2]')
    cookie.click()

    element = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/div[4]')
    WebDriverWait(driver, timeout=5)
    element.click()

    WebDriverWait(driver, timeout=10)

    github_commits = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]')
    github_stars = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]')
    github_forks = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]')
    github_contributors = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[4]/div[2]')
    github_followers = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[5]/div[2]')
    twitter_followers = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[6]/div[2]')
    reddit_members = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[7]/div[2]')

    list = [
        github_commits.text, github_stars.text, github_forks.text, 
        github_contributors.text, github_followers.text, twitter_followers.text, reddit_members.text
        ]

    list_new = [n.replace(',','') for n in list]
    
    crypto_project_info.append(list_new)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]"}
  (Session info: chrome=106.0.5249.103)
Stacktrace:
0   chromedriver                        0x000000010c0a9598 chromedriver + 4404632
1   chromedriver                        0x000000010c030fa3 chromedriver + 3911587
2   chromedriver                        0x000000010bcdde6a chromedriver + 425578
3   chromedriver                        0x000000010bd15c91 chromedriver + 654481
4   chromedriver                        0x000000010bd15e91 chromedriver + 654993
5   chromedriver                        0x000000010bd4d274 chromedriver + 881268
6   chromedriver                        0x000000010bd33dcd chromedriver + 777677
7   chromedriver                        0x000000010bd4af79 chromedriver + 872313
8   chromedriver                        0x000000010bd33bd3 chromedriver + 777171
9   chromedriver   

In [20]:
github_commits = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]')
github_stars = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]')
github_forks = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]')
github_contributors = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[4]/div[2]')
github_followers = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[5]/div[2]')
twitter_followers = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[6]/div[2]')
reddit_members = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[7]/div[2]')

list = [
    github_commits.text, github_stars.text, github_forks.text, 
    github_contributors.text, github_followers.text, twitter_followers.text, reddit_members.text
    ]

list_new = [n.replace(',','') for n in list]
np.array(list_new)

In [21]:
//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]
//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]


array(['12706', '4229', '1405', '97', '506', '2569415'], dtype='<U7')

In [53]:
launch_chrome(f'https://coinmarketcap.com/currencies/bitcoin/project-info/')
    
# driver.maximize_window()

# Cookie element blocks
cookie = driver.find_element(By.XPATH, '//*[@id="cmc-cookie-policy-banner"]/div[2]')
cookie.click()

element = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/div[4]')
WebDriverWait(driver, timeout=5)
element.click()
wait = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, '<div class="selected">Social Stats</div>'))
    )



github_commits = driver.find_element(By.TAG_NAME,'<div class="selected">Social Stats</div>')

print(github_commits.text)

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010b375598 chromedriver + 4404632
1   chromedriver                        0x000000010b2fcfa3 chromedriver + 3911587
2   chromedriver                        0x000000010afa9e6a chromedriver + 425578
3   chromedriver                        0x000000010afe1c91 chromedriver + 654481
4   chromedriver                        0x000000010afe1e91 chromedriver + 654993
5   chromedriver                        0x000000010b019274 chromedriver + 881268
6   chromedriver                        0x000000010afffdcd chromedriver + 777677
7   chromedriver                        0x000000010b016f79 chromedriver + 872313
8   chromedriver                        0x000000010afffbd3 chromedriver + 777171
9   chromedriver                        0x000000010afd4ced chromedriver + 601325
10  chromedriver                        0x000000010afd5e51 chromedriver + 605777
11  chromedriver                        0x000000010b3450ce chromedr

In [ ]:
driver.find_element(By.TAG_NAME, '<div class="selected">Social Stats</div>')
<div class="selected">Social Stats</div>